In [ ]:
import gradio as gr
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate

import os

os.environ["GRADIO_SERVER_PORT"] = "6666"
os.environ["GRADIO_SHARE"] = "True"

def init_ollama_llm(model, temperature, top_p):
    return Ollama(model=model,
                  temperature=temperature,
                  top_p=top_p,
                  callback=CallbackManager([StreamingStdOutCallbackHandler()])
                  )

def content_web(url):
    loader = WebBaseLoader(
        web_paths=(url,),
    )
    docs = loader.load()
    # chunk_overlap：分塊的重疊部分,重疊有助於降低將語句與與其相關的重要上下文分開的可能性，
    # 設置了chunk_overlap效果會更好
    # 合理的分詞會提高RAG的效果
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    return splits

def chroma_retriever_store_content(splits):
    # 基於ollama運行嵌入模型 nomic-embed-text ：A high-performing open embedding model with a large token context window.
    vectorstore = Chroma.from_documents(documents=splits,
                                        embedding=OllamaEmbeddings(model="bge-m3"))
    return vectorstore.as_retriever()

def rag_prompt():
    return PromptTemplate(
        input_variables=['context', 'question'],
        template=
        """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the 
        question. you don't know the answer, just say you don't know 
        without any explanation Question: {question} Context: {context} Answer:""",
    )

def ollama_rag_chroma_web_content(web_url, question,temperature,top_p):
    llm = init_ollama_llm('llama3.2:3b', temperature, top_p)
    splits = content_web(web_url)
    retriever = chroma_retriever_store_content(splits)
    qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever, chain_type_kwargs={"prompt": rag_prompt()})
    return qa_chain.invoke({"query": question})["result"]

demo = gr.Interface(
    fn=ollama_rag_chroma_web_content,
    inputs=[gr.Textbox(label="web_url",value="https://vuejs.org/guide/introduction.html",info="爬取內容的網頁地址"),
            "text",
            gr.Slider(0, 1,step=0.1),
            gr.Slider(0, 1,step=0.1)],
    outputs="text",
    title="Ollama+RAG Example",
    description="輸入網頁的URL，然後提問, 獲取答案"
)

demo.launch()